In [ ]:
# Install and Import Python Librairies 
!pip install alphabet-detector
!pip install pyarabic
!pip install arabic-reshaper
!pip install nltk
!pip install rapidfuzz
!pip install jarowinkler
!pip install jellyfish
!pip install xlsxwriter
import pandas
import numpy
import re
import hashlib
from alphabet_detector import AlphabetDetector
import pyarabic.araby as araby
import arabic_reshaper
import nltk
from nltk import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
nltk.download('punkt')
from rapidfuzz import process, fuzz
from rapidfuzz.process import cdist
from jarowinkler import jarowinkler_similarity
import jellyfish
import networkx as nx

In [3]:
# Normalization Libraries

def normalize_hamza(text):
    # Alif-Hamza
    text = re.sub("[إأٱآا]", "ا", text)
    # Waw-Hamza
    text = re.sub("ؤ", "ء", text)
    # Yay-Hamza
    text = re.sub("ئ", "ء", text)
    return text

def remove_accents(raw_text):
    raw_text = re.sub(u"[àáâãäå]", 'a', raw_text)
    raw_text = re.sub(u"[èéêë]", 'e', raw_text)
    raw_text = re.sub(u"[ìíîï]", 'i', raw_text)
    raw_text = re.sub(u"[òóôõö]", 'o', raw_text)
    raw_text = re.sub(u"[ùúûü]", 'u', raw_text)
    raw_text = re.sub(u"[ýÿ]", 'y', raw_text)
    raw_text = re.sub(u"ß", 'ss', raw_text)
    raw_text = re.sub(u"ñ", 'n', raw_text)
    raw_text = re.sub(u"ç", 'c', raw_text)
    return raw_text

def normalize(list_n):
    # Convert to String
    string = numpy.vectorize(str)
    list_n = string(list_n)
    # Lower case
    lower = numpy.vectorize(numpy.char.lower)
    list_n = lower(list_n)
    # Remove arabic diacritics
    diactrics = numpy.vectorize(araby.strip_diacritics)
    list_n = diactrics(list_n)
    # Remove arabic tatweel
    tatweel = numpy.vectorize(araby.strip_tatweel)
    list_n = tatweel(list_n)
    # Normalize types of hamza
    hamza = numpy.vectorize(normalize_hamza)
    list_n = hamza(list_n)
    # Reshape arabic text
    reshaper = numpy.vectorize(arabic_reshaper.reshape)
    list_n = reshaper(list_n)
    # Remove accents
    accents = numpy.vectorize(remove_accents)
    list_n = accents(list_n)
    # Remove punctuations
    punctuation = numpy.vectorize(re.sub)
    list_n = punctuation(r'\W', ' ', list_n)   
    # Remove numbers
    punctuation = numpy.vectorize(re.sub)
    list_n = punctuation(r'\d+', '', list_n)
    # Capitalize 
    capitalize = numpy.vectorize(str.title)
    list_n = capitalize(list_n)
    # Remove additional spaces
    space = numpy.vectorize(re.sub)
    list_n = space(' +', ' ', list_n)
    space = numpy.vectorize(str.lstrip)
    list_n = space(list_n)
    space = numpy.vectorize(str.rstrip)
    list_n = space(list_n)
    return list_n

In [4]:
# Tokenization Library

def tokenizer(name):
    name = TreebankWordDetokenizer().detokenize(sorted(word_tokenize(name))) 
    return name

In [5]:
# Transliteration Libraries

def load():
    # Read dictionary
    dictionary = pandas.read_csv("dictionary.csv", usecols=['arabic', 'roman'])
    arabic = numpy.array(dictionary['arabic'])
    roman = numpy.array(dictionary['roman'])
    # Define transliterator
    to_latin = {
        'ا': 'a', 'ء': 'a', 'إ': 'i', 'ى': 'a', 'بو': 'bou', 'ب': 'b', 'تو': 'tou', 'ت': 't', 'ثو': 'thou', 'ث': 'th',
        'جو': 'djou', 'ج': 'dj', 'حو': 'hou', 'حي': 'hy',
        'ح': 'h', 'خو': 'khou', 'خ': 'kh', 'دو': 'dou', 'د': 'd', 'ذو': 'dou', 'ذ': 'd', 'رو': 'rou', 'ري': 'ry',
        'ر': 'r', 'زو': 'zou', 'ز': 'z', 'سو': 'sou', 'س': 's',
        'شو': 'chou', 'ش': 'ch', 'صو': 'sou', 'ص': 's', 'ضو': 'dou', 'ض': 'd', 'طو': 'tou', 'ط': 't', 'ظو': 'dou',
        'ظ': 'd', 'غو': 'ghou', 'غ': 'gh', 'فو': 'fou', 'ف': 'f',
        'قو': 'qou', 'ق': 'q', 'كو': 'kou', 'ك': 'k', 'لو': 'lou', 'ل': 'l', 'مو': 'mou', 'م': 'm', 'نو': 'nou',
        'ن': 'n', 'هو': 'hou', 'ه': 'h', 'وو': 'wou', 'و': 'w',
        'يو': 'you', 'يي': 'yi', 'ي': 'i', 'ع': 'a', 'ة': 'a', 'ھ': 'h', 'ی': 'a', ' ':' '
    }
    return arabic, roman, to_latin

def translit_arabic(text, arabic, roman, to_latin):
    text = text.split(' ')
    for part in text:
        # Use dictionary
        if part in arabic:
            new = roman[arabic == part][0]
        # Use transliterator
        else:
            for k, v in to_latin.items():
                new = re.sub(k, v, part)
        text =list(map(lambda x: x.replace(part, new), text))
    text = ' '.join(text)
    return text

***Name Disambiguation***

In [7]:
def main(authors):
    # Load dictionary
    arabic, roman, to_latin = load()   
    # Normalization
    authors = normalize(authors)
    # Tokenization
    authors_r = [tokenizer(name) for name in authors]
    # Transliteration
    authors_r = [translit_arabic(name, arabic, roman, to_latin) for name in authors_r]
    # Jaro Winkler comparison
    distances = cdist(authors_r, authors_r, scorer=jarowinkler_similarity, score_cutoff=0.9, dtype=numpy.uint8, workers=-1)
    indx_l = numpy.stack(numpy.where(distances >= 0.9), axis = 1)
    # Match Rating comparison
    rates = numpy.array([jellyfish.jaro_distance(jellyfish.match_rating_codex(authors[indx[0]]),
                            jellyfish.match_rating_codex(authors[indx[1]])) for indx in indx_l])
    indx_l = indx_l[numpy.where(rates >= 0.9)]
    # Group similar
    g = nx.Graph()
    for p in indx_l:
        g.add_edges_from(zip(p, p[1:]))
    g = nx.connected_components(g)
    # Standardization
    for c in g:
        c = list(c)
        for i in range(len(c)):
            if authors[c[i]] != authors[c[0]]:
                print(authors[c[0]])
                authors[c[i]] = authors[c[0]]
    return authors